# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.
2. Append all this datasets and generate a unique dataframe. This newdataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**
3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.
4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

In [325]:
# Install the packages as required.
import os
import pickle
import pandas as pd
import numpy as np
import urllib.request
from datetime import datetime
import glob
import re

### Answer 1:

In [230]:
# 1. Read all the files contained in the specified path and save them in a list
path = r"../../_data\sbs\B_RawData\bancos"
file_list = [os.path.join(path, file) for file in os.listdir(path)]

In [660]:
# Iterate the reading of the files and save them in a dictionary named "excel_dict".
excel_dict = {} #Create empty dictionary

for file in file_list:
    df_tmp = pd.read_excel(file)
    # create column "date_info" with the day information
    df_tmp["date_info"] = os.path.splitext(file)[0][45:]
    # save all files in the dictionary according to their name
    file_name = os.path.splitext(file)[0][33:]
    excel_dict[file_name] = df_tmp

In [661]:
# Change the type of the "date_info" column to date 
for key, value in excel_dict.items():
    value["date_info"] = pd.to_datetime(value["date_info"], format="%d_%m_%Y")

In [662]:
# Check that the variable "date_info" was created in each dataframe
excel_dict["table_clean_26_2_2004"]["date_info"]

0    2004-02-26
1    2004-02-26
2    2004-02-26
3    2004-02-26
4    2004-02-26
5    2004-02-26
6    2004-02-26
7    2004-02-26
8    2004-02-26
9    2004-02-26
10   2004-02-26
11   2004-02-26
12   2004-02-26
13   2004-02-26
14   2004-02-26
15   2004-02-26
16   2004-02-26
Name: date_info, dtype: datetime64[ns]

### Answer 2:

In [679]:
#Check the different structures in the dataframes according the two first-columns names
results = []
for key, value in excel_dict.items():
    result = value.columns[0:2].tolist()
    results.append(result)

unique_results = list(set(tuple(result) for result in results))
for result in unique_results:
    print(list(result))

['Tasa Anual (%)', 'BBVA']
['Tasa Anual (%)', 'Continental']
['Unnamed: 0', 'COMERCIAL']
['Tasa Anual (%)', 'CRÉDITOS COMERCIALES']


In [707]:
# Homogenizar las estructuras tomando como base la de ['Tasa Anual (%)', 'BBVA'] por ser la más reciente

new_excel_dict = {} #Create a new empty dictionary

for key, value in excel_dict.items():
    
    # Primer caso: ['Unnamed: 0', 'COMERCIAL']
    if value.columns[1] == "COMERCIAL":
        # extraer la columna date_info
        date_info = value["date_info"]   
        # transponer df
        value = value.transpose().reset_index()
        # establecer los valores de la primera fila como headers del df
        value.columns = value.iloc[0]
        # extraer todas las filas excepto la primera ("Unnamed: 0") y la última ("date_info")
        value = value.iloc[1:-1,2:]
        # agregar la columna date_info
        value["date_info"] = date_info
        # rellenar los valores "NaN" de la columna date_info
        value["date_info"] = value['date_info'].fillna(value['date_info'].dropna().iloc[0])
        # renombrar columnas
        value.columns = [*map(lambda k: re.sub("BANCO DE |BANCO | BANCO| DEL PERU| BANK PERU| PERU|\(\*\)","",str(k)), value.columns)]
        value = value.rename(columns = {"nan": "Tasa Anual (%)", "BANBIF": "BIF", "AZTECA": "ALFIN", "Promedio del Sistema": "Promedio"})
        # Poner en mayúscula solo la primera letra de ciertos bancos
        value.columns = value.columns.str.capitalize()
        value = value.rename(columns = {"Tasa anual (%)": "Tasa Anual (%)",
                                        "Bbva":"BBVA", 
                                        "Bif":"BIF",
                                        "Bnp paribas el":"BNP Paribas", 
                                        "Hsbc":"HSBC", 
                                        "B santander central":"Santander",
                                        "Date_info": "date_info"})
        # guardar los resultados en el nuevo diccionario
        new_excel_dict[key] = value
        
    # Segundo caso: ['Tasa Anual (%)', 'CRÉDITOS COMERCIALES']    
    elif value.columns[1] == "CRÉDITOS COMERCIALES":
        # extraer la columna date_info
        date_info = value["date_info"] 
        # transponer df
        value = value.transpose().reset_index()
        # establecer los valores de la primera fila como headers del df
        value.columns = value.iloc[0]
        # extraer todas las filas excepto la primera ("Unnamed: 0") y la última ("date_info")
        value = value.iloc[1:-1,2:]
        # desplazar los valores de la segunda columna
        pos = list(value.iloc[:,1]).index("Tarjetas de Crédito")
        value.iloc[pos:,1:-1] = value.iloc[pos:,2:]
        # agregar la columna date_info
        value["date_info"] = date_info
        # rellenar los valores "NaN" de la columna date_info
        value["date_info"] = value['date_info'].fillna(value['date_info'].dropna().iloc[0])
        # renombrar columnas
        value.columns = [*map(lambda k: re.sub("BANCO DE |BANCO | BANCO| DEL PERU| BANK PERU| PERU|\(\*\)","",str(k)), value.columns)]
        value = value.rename(columns = {"BANBIF": "BIF", "AZTECA": "ALFIN", "Promedio del Sistema": "Promedio"})
        # Poner en mayúscula solo la primera letra de ciertos bancos
        value.columns = value.columns.str.capitalize()
        value = value.rename(columns = {"Tasa anual (%)": "Tasa Anual (%)",
                                        "Bbva": "BBVA", 
                                        "Bif": "BIF", 
                                        "Hsbc": "HSBC",
                                        "Date_info": "date_info"})
        # guardar los resultados en el nuevo diccionario
        new_excel_dict[key] = value
        
    # Tercer caso: ['Tasa Anual (%)', 'Continental']    
    else: 
        # renombrar columnas
        value = value.rename(columns = {"Crédito": "Credito", 
                                        "Continental": "BBVA", 
                                        "Financiero": "Pichincha",
                                        "Financiero  *": "Pichincha",
                                        "Azteca": "Alfin",
                                        "Azteca  *": "Alfin",
                                        "Banco GNB" : "GNB"})
        # guardar los resultados en el nuevo diccionario                     
        new_excel_dict[key] = value

In [709]:
# Unir todos los valores del diccionario "new_excel_dict
# Tomar como referencia el primer df del diccionario y concatenar el resto 
keys = list(new_excel_dict.keys())
first_df = new_excel_dict[keys[0]].copy()
# Modificar posición de las columnas
first_df = first_df[list(first_df.columns[[0,-1]])+list(first_df.columns[1:-1])]

for key in keys[1:]:
    first_df = pd.concat([first_df, new_excel_dict[key]], axis=0).replace("-",np.nan).replace("s.i.",np.nan)

# Eliminar las notas al final de los archivos excel
first_df = first_df[[*map(lambda k: not bool(re.search("^Nota", str(k))), first_df["Tasa Anual (%)"])]]  
first_df.drop("Nan", axis = "columns", inplace = True)

In [710]:
first_df

,Tasa Anual (%),date_info,Trabajo,BBVA,Comercio,Credito,Pichincha,BIF,Standard chartered,Sudamericano,...,Santander,Ripley,Alfin,Deutsche,GNB,ICBC,Bank of China,Cencosud,CAT,BCI
1,Avances en Cta.Corriente,2004-02-26,NaN,38.92,NaN,73.34,NaN,39.08,NaN,31.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sobregiros,2004-02-26,NaN,48.79,138.18,89.00,NaN,68.30,25.00,58.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dsctos. y préstamos hasta 30 días,2004-02-26,NaN,8.19,47.46,4.62,15.87,14.43,NaN,30.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dsctos. y préstamos 31 - 90 días,2004-02-26,NaN,5.94,21.61,7.19,15.71,10.91,NaN,11.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dsctos. y préstamos 91 - 180 días,2004-02-26,NaN,10.00,19.71,5.34,15.82,10.71,NaN,9.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Préstamos no Revolventes para libre disponibil...,2022-08-31,NaN,22.13,33.35,63.03,26.69,10.00,NaN,NaN,...,NaN,39.93,74.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Préstamos no Revolventes para libre disponibil...,2022-08-31,NaN,18.22,14.66,18.99,33.05,13.39,NaN,NaN,...,NaN,26.03,69.00,NaN,11.30,NaN,NaN,NaN,NaN,NaN
43,Créditos pignoraticios,2022-08-31,NaN,NaN,42.40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Hipotecarios,2022-08-31,NaN,9.28,9.08,9.27,8.72,9.23,NaN,NaN,...,NaN,NaN,NaN,NaN,8.03,NaN,NaN,NaN,NaN,NaN


### Answer 3:

In [711]:
first_df["Tasa Anual (%)"].unique()

array(['Avances en Cta.Corriente', 'Sobregiros',
       'Dsctos. y préstamos hasta 30 días',
       'Dsctos. y préstamos 31 - 90 días',
       'Dsctos. y préstamos 91 - 180 días',
       'Dsctos. y préstamos 181 - 360 días',
       'Dsctos. y préstamos más 360 días',
       'Dsctos. y préstamos hasta 360 días',
       'Tarjetas de crédito abiertas4', 'Tarjetas de crédito cerradas5',
       'Tarjetas de crédito',
       'Para préstamos menores o iguales a S/. 1800',
       'Para préstamos mayores a S/. 1800', 'Préstamos hasta 360 días',
       'Préstamos más de 360 días', 'HIPOTECARIO',
       'Avances en Cuenta Corriente', 'Descuentos',
       'Clientes deuda > S/. 5 MM4', 'Demás Clientes', 'Promedio',
       'Clientes deuda > S/. 5 MM', 'Tarjetas de Crédito',
       'Préstamos no revolventes', 'Para Automóviles',
       'Otros Préstamos Hasta 360 días',
       'Otros Préstamos a Más de 360 días', 'PRÉSTAMOS HIPOTECARIOS',
       'Corporativos', 'Préstamos hasta 30 días',
       'Prést

In [714]:
# Homogenizar valores
first_df["Tasa Anual (%)"] = first_df["Tasa Anual (%)"].replace(["Tarjetas de crédito abiertas4",
                                                                 "Tarjetas de crédito cerradas5",
                                                                 "Tarjetas de crédito", 
                                                                 "Tarjetas de Crdito"], 
                                                                 "Tarjetas de Crédito")

first_df["Tasa Anual (%)"] = first_df["Tasa Anual (%)"].replace(["Hipotecarios",
                                                                 "HIPOTECARIO",
                                                                 "PRÉSTAMOS HIPOTECARIOS"], 
                                                                 "Préstamos hipotecarios para vivienda")

# Crear variable año
first_df["year"] = [*map(lambda x: x.strftime("%Y"), first_df["date_info"])]

# Mantener los valores solicitados de la columna "Tasa Anual (%)"
five_top = first_df[first_df["Tasa Anual (%)"].isin(["Tarjetas de Crédito", "Préstamos hipotecarios para vivienda"])]

# Eliminar columnas que no se necesitan
five_top = five_top.drop(["date_info", "Promedio"], axis = "columns")

# Cambiar formato de "wide" a "long"
five_top = five_top.melt(id_vars = ["year", "Tasa Anual (%)"], var_name = "banks",value_name= "rate_value").dropna(subset=["rate_value"], axis = 0)

# Eliminar bancos sin información
five_top = five_top[five_top.rate_value != 's.i.']  

# Crear un nuevo df con el top 5
five_top.columns = ["year", "rate_concept", "banks", "rate_value"]
five_top = five_top.sort_values("rate_value", ascending = False).groupby(["year","rate_concept"]).head(5).reset_index(drop = True).sort_values(["year","rate_concept","rate_value"], ascending = [True, True, False])

five_top

,year,rate_concept,banks,rate_value
153,2001,Préstamos hipotecarios para vivienda,Comercio,8.24
154,2001,Préstamos hipotecarios para vivienda,Comercio,8.24
155,2001,Préstamos hipotecarios para vivienda,Pichincha,8.20
156,2001,Préstamos hipotecarios para vivienda,Pichincha,8.20
186,2001,Préstamos hipotecarios para vivienda,GNB,7.69
...,...,...,...,...
157,2022,Tarjetas de Crédito,Falabella,79.81
160,2022,Tarjetas de Crédito,Falabella,79.16
161,2022,Tarjetas de Crédito,Falabella,79.03
162,2022,Tarjetas de Crédito,Falabella,78.89


### Answer 4:

In [715]:
# Establecer ruta donde se guardarán los archivos
path = os.path.join("../../_output/sbs/", "group1") 
path

'../../_output/sbs/group1'

In [716]:
import shutil

In [717]:
# Si la ruta no existe, crear el directorio
if not os.path.exists(path):
    os.makedirs(path)
# Si la ruta existe, eliminar todo su contenido (incluyendo subdirectorios) usando shutil y crear nuevo directorio 
else:
    shutil.rmtree(path)           
    os.makedirs(path)

In [718]:
# crear lista con los nombres de los bancos, sin duplicados
banks_list = list(five_top.banks.unique())

# filtrar filas según banco, crear un excel por cada uno y guardar los archivos en la ruta determinada
for bank in banks_list:
    five_top.loc[five_top["banks"] == bank,:].to_excel(path + "/" + "table_" + bank.lower() +".xlsx")